##### submitted by Bikram Maharjan

# Midterm Certification Challenge: RAG

#### BUILD: Create a chatbot to communicate with NVIDIA 10-k Filings!


In [1]:
# DEPENDENCIES

!pip install -U -q langchain langchain-openai langchain_core langchain-community langchainhub openai

!pip install -qU faiss_cpu pymupdf tiktoken pandas

!pip install -qU ragas

# !pip install -qU wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 809.1/809.1 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 260.9/260.9 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 257.5/257.5 kB 20.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.6/67.6 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 20.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.8/77.8 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.5/138.5 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [44]:
# IMPORTS

import os
import getpass
from datasets import Dataset

import pandas as pd

import tiktoken

from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
from langchain.document_loaders import PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import ChatPromptTemplate
from langchain_community.vectorstores import FAISS
from langchain.retrievers import MultiQueryRetriever
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain


from operator import itemgetter

from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough


from ragas import evaluate
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    answer_correctness,
    context_recall,
    context_precision,
)

# import wandb

In [98]:
# Init OpenAI API
os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")

OpenAI API Key:··········


In [29]:
# START CODE

openai_chat_model = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")


In [36]:
# Helper function

def tiktoken_func(file):
    tokens = tiktoken.encoding_for_model("gpt-3.5-turbo").encode(
        file,
    )
    return len(tokens)


def process_file(file: str):

    pypdf_loader = PyMuPDFLoader(file)
    texts = pypdf_loader.load()

    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1400,
        chunk_overlap=150,
        length_function = tiktoken_func
    )

    documents = text_splitter.split_documents(texts)

    return documents

In [37]:
# NVIDIA 10-k Filings PDF file
documents = process_file(
        "https://d18rn0p25nwr6d.cloudfront.net/CIK-0001045810/1cbe8fe7-e08a-46e3-8dcc-b429fc06c1a4.pdf")

In [38]:
len(documents)

96

In [103]:
template = """

    Answer the question based only on the following context.

    If you cannot answer the question with the context, please respond with 'I don't know, can you provide more context?'.

    If the user question is not related to NVIDIA or NVIDIA 10-k Filings then please respond with "I can only help you with NVIDIA 10-k Filings report.\n\n".

    Always answer in full sentence.

    Always end your sentence with "\n\n\n Thank you for asking. What else can I help you regarding NVIDIA 10-k Filings?".

    Context:
    {context}

    Question:
    {input}
    """

prompt = ChatPromptTemplate.from_template(template)

# Utilizing FAISS vector store
vector_store = FAISS.from_documents(documents, embeddings)


In [40]:
# Create a retriever
retriever = vector_store.as_retriever()


In [42]:
# Test 1
# Invoke GPT to see relevant answer
retrieved_documents = retriever.invoke("Who is the E-VP, Operations - and how old are they?")

for doc in retrieved_documents:
  print(doc)

page_content="Table of Contents\nOur support is enhanced during times of crisis, such as war or economic volatility, to take care of our existing team of world-class talent and their families.\nDiversity, Inclusion, and Belonging\nWe believe that diverse teams fuel innovation, and we are committed to creating an inclusive culture that supports all employees.\nWhen recruiting for new talent or developing our current employees, we strive to build a diverse talent pipeline that includes those underrepresented in the\ntechnology field, including women, Black/African American, and Hispanic/Latino candidates.\nTo this end, we have been:\n•\nPartnering with institutions and professional organizations serving historically underrepresented communities;\n•\nEmbedding dedicated recruiting teams to business areas to shepherd underrepresented candidates through the interview process and find internal\nopportunities;\n•\nSupporting the development of women employees through programs aimed at buildin

In [43]:
# Test 2
# Invoke GPT to see relevant answer
retrieved_documents = retriever.invoke("What is the Total Number of Shares Purchased between December 25, 2023 - January 28, 2024?")

for doc in retrieved_documents:
  print(doc)

page_content='Table of Contents\napproximately 7 million shares for a total value of $2.8 billion through net share settlements. Refer to Note 4 of the Notes to the Consolidated Financial\nStatements in Part IV, Item 15 of this Annual Report on Form 10-K for further discussion regarding our equity incentive plans.\nStock Performance Graphs \nThe following graph compares the cumulative total shareholder return for our common stock, the S&P 500 Index, and the Nasdaq 100 Index for the five years\nended January 28, 2024. The graph assumes that $100 was invested on January 27, 2019 in our common stock and in each of the S&P 500 Index and the\nNasdaq 100 Index. Our common stock is a component of each of the presented indices. Total return assumes reinvestment of dividends in each of the indices\nindicated. Total return is based on historical results and is not intended to indicate future performance.\n*$100 invested on 1/27/19 in stock and in indices, including reinvestment of dividends.\nSo

In [47]:

# Baseline chain

retrieval_augmented_qa_chain = (
    # INVOKE CHAIN WITH: {"question" : "<>"}
    # "question" : populated by getting the value of the "question" key
    # "context"  : populated by getting the value of the "question" key and chaining it into the base_retriever
    {"context": itemgetter("question") | retriever, "input": itemgetter("question")}
    # "context"  : is assigned to a RunnablePassthrough object (will not be called or considered in the next step)
    #              by getting the value of the "context" key from the previous step
    | RunnablePassthrough.assign(context=itemgetter("context"))
    # "response" : the "context" and "question" values are used to format our prompt object and then piped
    #              into the LLM and stored in a key called "response"
    # "context"  : populated by getting the value of the "context" key from the previous step
    | {"response": prompt | openai_chat_model, "context": itemgetter("context")}
)

In [48]:

# Test 1
# Invoke GPT to see relevant answer

question = "Who is the E-VP, Operations - and how old are they?"

result = retrieval_augmented_qa_chain.invoke({"question" : question})

print(result["response"].content)


Debora Shoquist is the Executive Vice President of Operations, and she is 69 years old. 

Thank you for asking. What else can I help you regarding NVIDIA 10-k Filings?


In [49]:

# Test 2
# Invoke GPT to see relevant answer

question = "What is the gross carrying amount of Total Amortizable Intangible Assets for Jan 29, 2023?"

result = retrieval_augmented_qa_chain.invoke({"question" : question})

print(result["response"].content)


The gross carrying amount of Total Amortizable Intangible Assets for Jan 29, 2023 is $3,539 million.

Thank you for asking. What else can I help you regarding NVIDIA 10-k Filings?


### Evaluate with RAGAS!

In [51]:
# Initialize new document
eval_documents = documents

# Use document splitter for chunk
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1500,
    chunk_overlap = 100,
    length_function = tiktoken_func
)

eval_documents = text_splitter.split_documents(eval_documents)
len(eval_documents)


96

#### Synthetic Dataset Generation for Evaluation using Ragas

In [57]:
# Generate Synthetic dataset

from ragas.testset.generator import TestsetGenerator
from ragas.testset.evolutions import simple, reasoning, multi_context


# using non-depreciated method 'from_langchain' for testset generation
# generator = TestsetGenerator.from_langchain(
#     generator_llm = openai_chat_model,
#     critic_llm= "gpt-4",
#     embeddings = embeddings
#     )


generator = TestsetGenerator.with_openai()

testset = generator.generate_with_langchain_docs(eval_documents, test_size=10, distributions={simple: 0.2, reasoning: 0.4, multi_context: 0.4})



<ipython-input-57-f1e6045679c5>:15: DeprecationWarning: The function with_openai was deprecated in 0.1.4, and will be removed in the 0.2.0 release. Use from_langchain instead.
  generator = TestsetGenerator.with_openai()


embedding nodes:   0%|          | 0/244 [00:00<?, ?it/s]

Generating:   0%|          | 0/10 [00:00<?, ?it/s]

In [59]:
# review the generated synthetic dataset in a dataframe
testset.to_pandas()

,question,contexts,ground_truth,evolution_type,episode_done
0,What are some of the fields that benefit from ...,[Table of Contents\nResearchers and developers...,Some of the fields that benefit from NVIDIA's ...,simple,True
1,What are some key activities related to invest...,[Table of Contents\n•\ndevelop or acquire new ...,Some key activities related to investments in ...,simple,True
2,Which markets benefit from NVIDIA's computing ...,[Table of Contents\nResearchers and developers...,NVIDIA's computing solutions benefit the Data ...,reasoning,True
3,What processes and measures are in place for m...,[Table of Contents\nimmediately due and payabl...,The processes and measures in place for managi...,reasoning,True
4,What are the risks and consequences of defects...,[Table of Contents\ntraining data that may ori...,Defects and errors in product data and methods...,reasoning,True
5,How could tax changes impact a company's finan...,[Table of Contents\nData protection laws aroun...,Changes in tax laws or their interpretation by...,reasoning,True
6,What are the risks of not meeting industry nee...,[Table of Contents\n•\nInternational sales and...,Our accelerated computing platforms experience...,multi_context,True
7,"Which industries use AI, NVIDIA GPUs, and soft...","[ative AI-enabled products and services,\nor t...","The financial services industry uses AI, NVIDI...",multi_context,True
8,What are the potential risks to a business fro...,[Table of Contents\ntraining data that may ori...,The potential risks to a business from uncerta...,multi_context,True
9,What are the potential negative effects of tax...,[Table of Contents\nData protection laws aroun...,Tax liabilities can have potential negative ef...,multi_context,True


In [60]:

# Save the generated df, just in case...
testset.to_pandas().to_csv('nvidia-synthetic-dataset-ragas.csv', index=False)


In [67]:

# Convert the generated testset into Pandas DF
test_df = testset.to_pandas()

# Convert into list for distribution
test_questions = test_df["question"].values.tolist()
test_groundtruths = test_df["ground_truth"].values.tolist()
test_evolution_type = test_df["evolution_type"].values.tolist()


In [71]:
test_questions

["What are some of the fields that benefit from NVIDIA's computing solutions and their applications?",
 'What are some key activities related to investments in research and development and new product development?',
 "Which markets benefit from NVIDIA's computing solutions in molecular dynamics simulation, climate forecasting, wind tunnel simulation, and genomics?",
 'What processes and measures are in place for managing and assessing cybersecurity risks, and who oversees these efforts?',
 'What are the risks and consequences of defects and errors in product data and methods, and how do they impact our business?',
 "How could tax changes impact a company's financial position and results?",
 'What are the risks of not meeting industry needs, laws, regulations, and political actions?',
 'Which industries use AI, NVIDIA GPUs, and software for customer support, fraud detection, drug discovery, and critical workflows?',
 'What are the potential risks to a business from uncertainty, security

In [78]:

# Pre-Processing for mertic analysis / Generate using RAG Pipeline
answers = []
contexts = []

# invoke baseline metrics
for question in test_questions:
  print("Processing question:", question)
  response = retrieval_augmented_qa_chain.invoke({"question" : question})
  answers.append(response["response"].content)
  contexts.append([context.page_content for context in response["context"]])


# sample output
result["response"].content


Processing question: What are some of the fields that benefit from NVIDIA's computing solutions and their applications?
Processing question: What are some key activities related to investments in research and development and new product development?
Processing question: Which markets benefit from NVIDIA's computing solutions in molecular dynamics simulation, climate forecasting, wind tunnel simulation, and genomics?
Processing question: What processes and measures are in place for managing and assessing cybersecurity risks, and who oversees these efforts?
Processing question: What are the risks and consequences of defects and errors in product data and methods, and how do they impact our business?
Processing question: How could tax changes impact a company's financial position and results?
Processing question: What are the risks of not meeting industry needs, laws, regulations, and political actions?
Processing question: Which industries use AI, NVIDIA GPUs, and software for customer s

'The gross carrying amount of Total Amortizable Intangible Assets for Jan 29, 2023 is $3,539 million.\n\nThank you for asking. What else can I help you regarding NVIDIA 10-k Filings?'

In [ ]:

# Convert it into a dataset
response_dataset = Dataset.from_dict({
    "question" : test_questions,
    "answer" : answers,
    "contexts" : contexts,
    "ground_truth" : test_groundtruths,
    "evolution_type" : test_evolution_type

})


In [83]:

# Prepare RAGAS metric pipeline list
metrics = [
    faithfulness,
    answer_relevancy,
    context_recall,
    context_precision,
    answer_correctness,
]

In [85]:

# Evaluate our baseline
results = evaluate(response_dataset, metrics)
results


{'faithfulness': 0.7500, 'answer_relevancy': 0.9736, 'context_recall': 1.0000, 'context_precision': 0.9417, 'answer_correctness': 0.6510}

In [101]:

# Baseline metric
pd.DataFrame(list(results.items()), columns=['Metric', 'Baseline'])


,Metric,Baseline
0,faithfulness,0.750000
1,answer_relevancy,0.973636
2,context_recall,1.000000
3,context_precision,0.941667
4,answer_correctness,0.651045


**Explanation**: In assessing the baseline model's performance across various metrics, several observations emerge. Notably, it demonstrates moderate faithfulness (0.75) to the ground truth and high relevancy (0.973) to posed questions. Moreover, it exhibits perfect context recall (1.0), indicating its ability to retrieve relevant information comprehensively. While context precision (0.942) is commendable, there's room for improvement in answer correctness (0.651), suggesting potential enhancements for accuracy.

In [86]:

# Convert the baseline result into df
results_df = results.to_pandas()
results_df.head() # first 5 rows


,question,answer,contexts,ground_truth,evolution_type,faithfulness,answer_relevancy,context_recall,context_precision,answer_correctness
0,What are some of the fields that benefit from ...,Some of the fields that benefit from NVIDIA's ...,[Table of Contents\nResearchers and developers...,Some of the fields that benefit from NVIDIA's ...,simple,NaN,0.979655,1.0,1.0,0.593013
1,What are some key activities related to invest...,Key activities related to investments in resea...,[Table of Contents\n•\ndevelop or acquire new ...,Some key activities related to investments in ...,simple,1.00,0.996284,1.0,1.0,0.803573
2,Which markets benefit from NVIDIA's computing ...,The markets that benefit from NVIDIA's computi...,[Table of Contents\nResearchers and developers...,NVIDIA's computing solutions benefit the Data ...,reasoning,0.25,0.981377,1.0,1.0,0.605749
3,What processes and measures are in place for m...,The processes and measures in place for managi...,[Table of Contents\nimmediately due and payabl...,The processes and measures in place for managi...,reasoning,NaN,0.970047,1.0,1.0,0.378509
4,What are the risks and consequences of defects...,The risks and consequences of defects and erro...,[Table of Contents\ntraining data that may ori...,Defects and errors in product data and methods...,reasoning,NaN,0.978879,1.0,1.0,0.752851


### Adjust our RAG pipeline with advance chain

In [91]:

# We will use MultiQueryRetriever to validate improvements
advanced_retriever = MultiQueryRetriever.from_llm(
    retriever=retriever, llm=openai_chat_model)

# Re create RAG Pipeline
document_chain = create_stuff_documents_chain(openai_chat_model, prompt)

advanced_retrieval_chain = create_retrieval_chain(advanced_retriever, document_chain)


In [92]:

# Question 1

response = advanced_retrieval_chain.invoke({"input":  "Who is the E-VP, Operations - and how old are they?"})

llm_answer = response['answer']

print("llm_answer: ", llm_answer)


llm_answer:  Debora Shoquist is the Executive Vice President of Operations, and she is 69 years old. 

Thank you for asking. What else can I help you regarding NVIDIA 10-k Filings?


In [93]:

# Question 2

response = advanced_retrieval_chain.invoke({"input":  "What is the gross carrying amount of Total Amortizable Intangible Assets for Jan 29, 2023?"})

llm_answer = response['answer']

print("llm_answer: ", llm_answer)


llm_answer:  The gross carrying amount of Total Amortizable Intangible Assets for Jan 29, 2023, was $3,539 million. 

Thank you for asking. What else can I help you regarding NVIDIA 10-k Filings?


#Additional Component

### Validate Generated Testset

In [104]:

# Pre-Processing for mertic analysis
answers = []
contexts = []

for question in test_questions:
  print("Processing question:", question)
  response = advanced_retrieval_chain.invoke({"input" : question})
  answers.append(response["answer"])
  contexts.append([context.page_content for context in response["context"]])


# Convert it into a dataset
response_dataset_advanced_retrieval = Dataset.from_dict({
    "question" : test_questions,
    "answer" : answers,
    "contexts" : contexts,
    "ground_truth" : test_groundtruths,
    "evolution_type" : test_evolution_type

})


Processing question: What are some of the fields that benefit from NVIDIA's computing solutions and their applications?
Processing question: What are some key activities related to investments in research and development and new product development?
Processing question: Which markets benefit from NVIDIA's computing solutions in molecular dynamics simulation, climate forecasting, wind tunnel simulation, and genomics?
Processing question: What processes and measures are in place for managing and assessing cybersecurity risks, and who oversees these efforts?
Processing question: What are the risks and consequences of defects and errors in product data and methods, and how do they impact our business?
Processing question: How could tax changes impact a company's financial position and results?
Processing question: What are the risks of not meeting industry needs, laws, regulations, and political actions?
Processing question: Which industries use AI, NVIDIA GPUs, and software for customer s

In [99]:

# Prepare metric pipeline list
metrics = [
    faithfulness,
    answer_relevancy,
    context_recall,
    context_precision,
    answer_correctness,
]


# Evaluate the result
advanced_retrieval_results = evaluate(response_dataset_advanced_retrieval, metrics)

# Show the overall metrics summary
advanced_retrieval_results


Evaluating:   0%|          | 0/50 [00:00<?, ?it/s]

{'faithfulness': 0.8500, 'answer_relevancy': 0.9451, 'context_recall': 0.9500, 'context_precision': 0.9538, 'answer_correctness': 0.6454}

In [ ]:

# Review the dataframe
advanced_retrieval_results.to_pandas()


,question,answer,contexts,ground_truth,evolution_type,faithfulness,answer_relevancy,context_recall,context_precision,answer_correctness
0,What are some of the risks and challenges asso...,Some of the risks and challenges associated wi...,"[governmental, regulatory approval or restrict...",Some of the risks and challenges associated wi...,simple,1.00,1.000000,0.75,0.916667,0.326682
1,How do excessive or shifting export controls i...,Excessive or shifting export controls can harm...,[serve markets worldwide. Excessive or shiftin...,Excessive or shifting export controls may harm...,simple,NaN,0.974689,1.00,1.000000,0.742871
2,What challenges can arise when integrating tec...,Challenges that can arise when integrating tec...,"[target company, technology may evolve and mak...",Challenges that can arise when integrating tec...,reasoning,1.00,0.977172,1.00,1.000000,0.425246
3,What tools does the company use to assess empl...,"The company uses pulse surveys, a suggestion b...",[readiness for future advancement.\nTo evaluat...,"The company uses pulse surveys, a suggestion b...",reasoning,0.80,0.940681,1.00,1.000000,0.603051
4,How does the unified architecture support dive...,The unified architecture supports diverse mark...,[serve. While the computing requirements of th...,The programmable nature of our architecture al...,reasoning,0.75,0.893298,1.00,0.950000,0.646074
5,What was NVIDIA's renewable energy percentage ...,NVIDIA's renewable energy percentage in fiscal...,[Table of Contents\nor generate enough renewab...,"In fiscal year 2023, NVIDIA's renewable energy...",reasoning,1.00,0.962348,1.00,1.000000,0.612359
6,What challenges and negative outcomes can aris...,Challenges and negative outcomes from acquisit...,"[governmental, regulatory approval or restrict...",Challenges and negative outcomes that can aris...,multi_context,1.00,0.963360,1.00,1.000000,0.645582
7,What are the risks of using algorithms and thi...,The risks of using algorithms and third-party ...,"[algorithms. These datasets may be flawed, ins...",The risks of using algorithms and third-party ...,multi_context,1.00,0.999992,1.00,1.000000,0.591192
8,What products and thresholds are subject to ex...,The products and thresholds subject to export ...,"[regions, including some countries in the Midd...",The products and thresholds subject to export ...,multi_context,1.00,0.948962,1.00,1.000000,0.619322
9,How is DRIVE being introduced to the market th...,DRIVE is being introduced to the market throug...,[a wide range of intelligent functions into a ...,We are bringing DRIVE to the market through ou...,multi_context,1.00,0.923905,1.00,1.000000,0.658314


In [100]:

pd.DataFrame(list(advanced_retrieval_results.items()), columns=['Metric', 'Baseline'])


,Metric,Baseline
0,faithfulness,0.850000
1,answer_relevancy,0.945141
2,context_recall,0.950000
3,context_precision,0.953778
4,answer_correctness,0.645435


**Explanation**: In the evaluation above, my model demonstrates its proficiency across several key metrics. It achieves a high level of faithfulness to ground truth (0.85) and relevancy to questions (0.945). Additionally, it excels in recalling (0.95) and precisely utilizing context (0.954), though it still has room for improvement in answer correctness (0.645).




In [102]:

# Compare the metrics
# Create a dataframe

df_original = pd.DataFrame(list(results.items()), columns=['Metric', 'Baseline'])
df_comparison = pd.DataFrame(list(advanced_retrieval_results.items()), columns=['Metric', 'MultiQueryRetriever with Document Stuffing'])

df_merged = pd.merge(df_original, df_comparison, on='Metric')

df_merged['Delta'] = df_merged['MultiQueryRetriever with Document Stuffing'] - df_merged['Baseline']

df_merged


,Metric,Baseline,MultiQueryRetriever with Document Stuffing,Delta
0,faithfulness,0.750000,0.850000,0.100000
1,answer_relevancy,0.973636,0.945141,-0.028495
2,context_recall,1.000000,0.950000,-0.050000
3,context_precision,0.941667,0.953778,0.012111
4,answer_correctness,0.651045,0.645435,-0.005610


### **Conclusion**

In comparing my model's performance to the baseline, several noteworthy improvements and shortcomings are evident. While my model exhibits enhanced faithfulness (0.85) compared to the baseline (0.75), the answer relevancy (0.945) and context recall (0.95) are slightly lower. However, there are improvements in context precision (0.954) and minor declines in answer correctness (0.645). These insights guide further optimization efforts to enhance overall performance.

The reason why I think its not so great at getting the answers wright is due to limited training data I set since it keep timing out over large dataset. Other reason could be ambiguity in the language.

